## 資料處理-SYDNEY

In [ ]:
import os
import pandas as pd
import re
import numpy as np
# from tabulate import tabulate  # 暫時註解，如果不需要可以移除

# 分類出發時間
def classify_departure_time_period(departure_time):
    if "凌晨" in departure_time or "清晨" in departure_time:
        hour = int(departure_time.split("清晨")[-1].split(":")[0]) if "清晨" in departure_time else int(departure_time.split("凌晨")[-1].split(":")[0])
        if hour == 12 or 0 <= hour < 6:
            return "凌晨班機"
    if "清晨" in departure_time or "早上" in departure_time or "上午" in departure_time:
        hour = int(departure_time.split(":")[0].split("清晨")[-1].split("早上")[-1].split("上午")[-1])
        if 6 <= hour < 9:
            return "早晨班機"
        elif 9 <= hour < 12:
            return "上午班機"
    if "中午" in departure_time:
        hour = int(departure_time.split("中午")[-1].split(":")[0])
        if hour == 12:
            return "下午班機"
    if "下午" in departure_time:
        hour = int(departure_time.split("下午")[-1].split(":")[0]) + 12
        if 12 <= hour < 18:
            return "下午班機"
    if "晚上" in departure_time or "下午" in departure_time:
        hour = int(departure_time.split("晚上")[-1].split("下午")[-1].split(":")[0]) + 12
        if 18 <= hour < 24:
            return "晚間班機"
    return "未分類"

# 分類抵達時間
def classify_arrival_time_period(arrival_time):
    if "凌晨" in arrival_time or "清晨" in arrival_time:
        hour = int(arrival_time.split("清晨")[-1].split(":")[0]) if "清晨" in arrival_time else int(arrival_time.split("凌晨")[-1].split(":")[0])
        if hour == 12 or 0 <= hour < 6:
            return "凌晨抵達"
    if "清晨" in arrival_time or "早上" in arrival_time or "上午" in arrival_time:
        hour = int(arrival_time.split(":")[0].split("清晨")[-1].split("早上")[-1].split("上午")[-1])
        if 6 <= hour < 9:
            return "早晨抵達"
        elif 9 <= hour < 12:
            return "上午抵達"
    if "中午" in arrival_time:
        hour = int(arrival_time.split("中午")[-1].split(":")[0])
        if hour == 12:
            return "下午抵達"
    if "下午" in arrival_time:
        hour = int(arrival_time.split("下午")[-1].split(":")[0]) + 12
        if 12 <= hour < 18:
            return "下午抵達"
    if "晚上" in arrival_time or "下午" in arrival_time:
        hour = int(arrival_time.split("晚上")[-1].split("下午")[-1].split(":")[0]) + 12
        if 18 <= hour < 24:
            return "晚間抵達"
    return "未分類"

# 剔除離群值（依需求啟用）
# def remove_outliers(df, column):
#     mean = df[column].mean()
#     std = df[column].std()
#     lower_bound = mean - 3 * std
#     upper_bound = mean + 3 * std
#     clean_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
#     print(f"剔除離群值前資料筆數：{len(df)}, 剔除離群值後資料筆數：{len(clean_df)}")
#     return clean_df

# 機型分類：0 窄體機, 1 寬體機, 2 超大型客機, 其他 -1
def classify_aircraft_type(aircraft_model):
    if pd.isnull(aircraft_model):
        return -1
    elif any(x in aircraft_model for x in ['Airbus A320', 'Airbus A321neo', 'Airbus A320neo', 'Boeing 737']):
        return 0
    elif any(x in aircraft_model for x in ['Boeing 787', 'Boeing 787-10', 'Airbus A330', 'Airbus A330-900neo', 'Airbus A350', 'Boeing 777', 'Boeing 767']):
        return 1
    elif any(x in aircraft_model for x in ['Airbus A380', 'Boeing 747']):
        return 2
    else:
        return -1

# 主處理與合併函數，透過參數化支援不同來源與艙等篩選邏輯
def process_and_merge_files(start_date, end_date, base_path, output_path, file_pattern, cabin_class):
    """
    Parameters:
      start_date: 字串，如 '1021'，代表 MMDD
      end_date: 字串，如 '0320'
      base_path: 資料存放目錄
      output_path: 輸出 CSV 的完整路徑
      file_pattern: 檔案名稱格式，內含一個 {} 以放置日期字串，例如 "sydney_{}.csv" 或 "sydney_business_{}.csv"
      cabin_class: 'economy' 表示僅保留艙等為「經濟艙」；'business' 表示僅保留艙等為「商務艙」
    """
    # 航空聯盟分類
    star_alliance = ['長榮航空', '全日空航空', '新加坡航空', '韓亞航空', '紐西蘭航空', 'THAI']
    skyteam = ['中華航空', '中國東方航空', '越南航空', '廈門航空', '大韓航空', '加魯達印尼航空']
    oneworld = ['國泰航空', '澳洲航空', '日本航空', '馬來西亞航空']
    value_alliance = ['酷航', '宿霧太平洋航空']
    
    # 建立日期範圍（跨年度注意起始月份判斷）
    dates = pd.date_range(start=f'2024-{start_date[:2]}-{start_date[2:]}', 
                          end=f'2025-{end_date[:2]}-{end_date[2:]}')
    date_strings = [date.strftime('%m%d') for date in dates]

    data_frames = []
    found_files = 0
    processed_files = 0
    
    print(f"開始處理，日期範圍：{start_date} 到 {end_date}")
    print(f"基礎路徑：{base_path}")
    print(f"檔案格式：{file_pattern}")
    print(f"艙等篩選：{cabin_class}")
    
    for date in date_strings:
        file_path = os.path.join(base_path, file_pattern.format(date))
        if os.path.exists(file_path):
            found_files += 1
            month, day = int(date[:2]), int(date[2:])
            year = 2024 if month >= 10 else 2025
            file_date = pd.Timestamp(year=year, month=month, day=day)
            try:
                df = pd.read_csv(file_path)
                if not df.empty:
                    df = df.dropna(how="all")
                    # 格式化出發日期（僅取年月日）
                    df['出發日期'] = pd.to_datetime(df['出發日期'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]).dt.strftime('%Y-%m-%d')
                    # 新增星期欄位
                    df['星期'] = pd.to_datetime(df['出發日期']).dt.day_name(locale='zh_TW')
                    # 處理價格欄位：移除空值、符號後轉換為數字
                    df = df.dropna(subset=['價格'])
                    df = df[df['價格'].str.match(r'^[NT\$,\d\s]+$')]
                    df['價格'] = df['價格'].replace(r'[NT\$,\s]', '', regex=True).astype(int)
                    # 轉換出發日期並計算 day left（檔案日期與出發日期的差值）
                    df['出發日期'] = pd.to_datetime(df['出發日期'])
                    df['day left'] = (df['出發日期'] - file_date).dt.days

                    # 內部函數：解析飛行時間字串（轉換為總分鐘數）
                    def parse_duration(duration_str):
                        hours, minutes = 0, 0
                        if "小時" in duration_str:
                            hours = int(duration_str.split("小時")[0].strip())
                            duration_str = duration_str.split("小時")[1]
                        if "分鐘" in duration_str:
                            minutes = int(duration_str.split("分鐘")[0].strip())
                        return hours * 60 + minutes

                    # 內部函數：處理主要段落（航空公司或艙等）
                    def process_main_segment(row, column_name):
                        value = row[column_name]
                        if not isinstance(value, str):
                            return value
                        if value == "亞洲航空 X 亞洲航空 X":
                            return "亞洲航空 X"
                        value = " ".join(value.split())
                        unique_segments = set(value.split(" "))
                        if len(unique_segments) == 1:
                            return value.split(" ")[0]
                        segments = value.split(" ")
                        if len(segments) == 2:
                            try:
                                first_duration = parse_duration(row["第一段飛行時間"])
                                second_duration = parse_duration(row["第二段飛行時間"])
                                return segments[0] if first_duration >= second_duration else segments[1]
                            except Exception:
                                return None
                        return value

                    # 內部函數：處理次要段落（機型或航班代碼）
                    def process_second_segment(row, column_name):
                        value = row[column_name]
                        if not isinstance(value, str):
                            return value
                        value = " ".join(value.split())
                        segments = re.findall(r'[^\s]+\s[^\s]+', value)
                        if len(segments) != 2:
                            return value
                        try:
                            durations = [parse_duration(row["第一段飛行時間"]), parse_duration(row["第二段飛行時間"])]
                            return segments[0] if durations[0] >= durations[1] else segments[1]
                        except Exception:
                            return None

                    # 處理航空公司、機型、航班代碼與艙等的主要段落
                    df["航空公司（主航段）"] = df.apply(lambda row: process_main_segment(row, "航空公司"), axis=1)
                    df["機型（主航段）"] = df.apply(lambda row: process_second_segment(row, "機型"), axis=1)
                    df["航班代碼（主航段）"] = df.apply(lambda row: process_second_segment(row, "航班代碼"), axis=1)
                    df["艙等（主航段）"] = df.apply(lambda row: process_main_segment(row, "艙等"), axis=1)
                    
                    # 新增航空公司組合欄位：判斷前後航空公司是否一致
                    def airline_combination(row):
                        if not isinstance(row["航空公司"], str):
                            return 0
                        segments = row["航空公司"].split(" ")
                        if len(segments) == 2:
                            return 1 if segments[0] == segments[1] else 0
                        return 0
                    df["航空公司組合"] = df.apply(airline_combination, axis=1)
                    
                    # 根據航空公司分配航空聯盟
                    def assign_alliance(airline):
                        if airline in star_alliance:
                            return 1
                        elif airline in skyteam:
                            return 2
                        elif airline in oneworld:
                            return 3
                        elif airline in value_alliance:
                            return 4
                        elif airline in ['菲律賓航空', '中國南方航空']:
                            return 5
                        elif airline in ['越捷航空', '亞洲航空 X', '捷星航空']:
                            return 6
                        else:
                            return 0
                    
                    # 調整欄位順序：將主航段相關欄位移至對應欄位後
                    cols = df.columns.tolist()
                    for col in ["航空公司（主航段）", "機型（主航段）", "航班代碼（主航段）", "艙等（主航段）"]:
                        cols.insert(cols.index(col.replace("（主航段）", "")) + 1, cols.pop(cols.index(col)))
                    df = df[cols]
                    
                    # 新增航空聯盟欄位
                    df['航空聯盟'] = df['航空公司（主航段）'].apply(assign_alliance)
                    cols = df.columns.tolist()
                    cols.insert(cols.index("航空公司（主航段）") + 1, cols.pop(cols.index("航空聯盟")))
                    df = df[cols]
                    
                    # 將星期欄位移到出發時間之後
                    cols = df.columns.tolist()
                    cols.insert(cols.index('出發時間') + 1, cols.pop(cols.index('星期')))
                    df = df[cols]
                    
                    # 將航空公司組合移至航空公司（主航段）之後
                    cols = df.columns.tolist()
                    cols.insert(cols.index("航空公司（主航段）") + 1, cols.pop(cols.index("航空公司組合")))
                    df = df[cols]
                    
                    # 新增出發與抵達時段
                    df['出發時段'] = df['出發時間'].apply(classify_departure_time_period)
                    cols = df.columns.tolist()
                    if '出發時間' in cols and '出發時段' in cols:
                        cols.insert(cols.index('出發時間') + 1, cols.pop(cols.index('出發時段')))
                    df = df[cols]
                    
                    df['抵達時段'] = df['抵達時間'].apply(classify_arrival_time_period)
                    cols = df.columns.tolist()
                    if '抵達時間' in cols and '抵達時段' in cols:
                        cols.insert(cols.index('抵達時間') + 1, cols.pop(cols.index('抵達時段')))
                    df = df[cols]
                    
                    # 新增航空聯盟組合欄位：判斷兩段航空公司所屬聯盟是否一致
                    def airline_alliance_combination(row):
                        if not isinstance(row["航空公司"], str):
                            return 0
                        segments = row["航空公司"].split(" ")
                        if len(segments) == 1:
                            return 1
                        if len(segments) == 2:
                            return 1 if assign_alliance(segments[0]) == assign_alliance(segments[1]) else 0
                        return 0
                    df["航空聯盟組合"] = df.apply(airline_alliance_combination, axis=1)
                    cols = df.columns.tolist()
                    if "航空公司（主航段）" in cols and "航空聯盟組合" in cols:
                        cols.insert(cols.index("航空公司（主航段）") + 1, cols.pop(cols.index("航空聯盟組合")))
                    df = df[cols]
                    
                    processed_files += 1
                    data_frames.append(df)
                else:
                    print(f"  警告：檔案 {file_path} 為空")
            except Exception as e:
                print(f"  錯誤：讀取檔案 {file_path} 時發生錯誤：{e}")
        # else:
        #     print(f"  檔案不存在：{file_path}")
    
    print(f"找到 {found_files} 個檔案，成功處理 {processed_files} 個檔案")
    
    # 合併所有資料
    merged_data = pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()
    if merged_data.empty:
        print("沒有合併到任何資料。")
        print(f"可能的原因：")
        print(f"  1. 路徑不正確：{base_path}")
        print(f"  2. 檔案格式不匹配：{file_pattern}")
        print(f"  3. 日期範圍內沒有對應的檔案")
        return
    
    # 處理 '停靠站數量' 欄位，若為字串則依照內容轉換為數字
    if merged_data['停靠站數量'].dtype == 'object':
        merged_data['停靠站數量'] = merged_data['停靠站數量'].apply(lambda x: 1 if x == '需轉機 1 次的航班。' else x)
        merged_data['停靠站數量'] = merged_data['停靠站數量'].apply(lambda x: 0 if x == '直達航班。' else x)
    
    # 新增機型分類欄位，並調整至機型（主航段）後面
    merged_data['機型分類'] = merged_data['機型（主航段）'].apply(classify_aircraft_type)
    cols = merged_data.columns.tolist()
    if '機型分類' in cols and '機型（主航段）' in cols:
        cols.insert(cols.index('機型（主航段）') + 1, cols.pop(cols.index('機型分類')))
    merged_data = merged_data[cols]
    
    # 根據 cabin_class 篩選艙等：
    # "economy" 只保留「經濟艙」
    # "business" 只保留「商務艙」
    if cabin_class == "economy":
        merged_data = merged_data[merged_data['艙等（主航段）'] == '經濟艙']
    elif cabin_class == "business":
        merged_data = merged_data[merged_data['艙等（主航段）'] == '商務艙']
    
    # 計算每個航班的筆數
    flight_counts = merged_data.groupby(['出發日期', '航班代碼', '停留時間']).size().reset_index(name='筆數')

    # 統計有多少筆被剔除
    original_flight_count = flight_counts.shape[0]
    valid_flights = flight_counts[flight_counts['筆數'] >= 35]
    removed_flight_count = original_flight_count - valid_flights.shape[0]
    print(f"原始航班數量：{original_flight_count}，刪除少於 35 筆的航班數量：{removed_flight_count}")

    # 分組統計：平均價格、最低價格、中位數價格與價格變異
    grouped = merged_data.groupby(['出發日期', '航班代碼', '停留時間'], as_index=False).agg(
        平均價格=('價格', 'mean'),
        最低價格=('價格', 'min'),
        中位數價格=('價格', 'median'),
        價格變異=('價格', 'var')
    )
    grouped['平均價格'] = grouped['平均價格'].round(2).astype(int)
    grouped['最低價格'] = grouped['最低價格'].round(2).astype(int)
    grouped['中位數價格'] = grouped['中位數價格'].round(2).astype(int)
    grouped['價格變異'] = grouped['價格變異'].fillna(0).round(2)
    
    # 新增「隨機購買平均價格」：將91天分成三段（前30、中30、後31），每段隨機抽取4筆價格並計算平均
    def calculate_segmented_random_price(group):
        """
        將每組資料按 day left 分成三段（總共91天）：
        - 前段：day left 最大的30天（較早購買，day left 61-90）
        - 中段：day left 中間的30天（day left 31-60）
        - 後段：day left 最小的31天（接近出發，day left 0-30）
        從每段隨機抽取4筆價格計算平均，返回三個獨立的平均值
        """
        prices = group['價格'].values
        day_lefts = group['day left'].values
        
        # 分段：根據 day left 範圍分成三段（假設總共91天）
        # 前段：day left 61-90（較早購買的31天）
        front_segment = prices[(day_lefts >= 61) & (day_lefts <= 90)]
        # 中段：day left 31-60（中間的30天）
        middle_segment = prices[(day_lefts >= 31) & (day_lefts <= 60)]
        # 後段：day left 0-30（接近出發的30天）
        back_segment = prices[(day_lefts >= 0) & (day_lefts <= 30)]
        
        # 前段：隨機抽取4筆（如果不足4筆則全部使用）
        if len(front_segment) > 0:
            n_samples = min(4, len(front_segment))
            front_mean = front_segment[np.random.choice(len(front_segment), n_samples, replace=False)].mean()
            front_mean = round(front_mean, 2)
        else:
            front_mean = None
        
        # 中段：隨機抽取4筆（如果不足4筆則全部使用）
        if len(middle_segment) > 0:
            n_samples = min(4, len(middle_segment))
            middle_mean = middle_segment[np.random.choice(len(middle_segment), n_samples, replace=False)].mean()
            middle_mean = round(middle_mean, 2)
        else:
            middle_mean = None
        
        # 後段：隨機抽取4筆（如果不足4筆則全部使用）
        if len(back_segment) > 0:
            n_samples = min(4, len(back_segment))
            back_mean = back_segment[np.random.choice(len(back_segment), n_samples, replace=False)].mean()
            back_mean = round(back_mean, 2)
        else:
            back_mean = None
        
        return {
            '前段隨機購買平均價格': front_mean,
            '中段隨機購買平均價格': middle_mean,
            '後段隨機購買平均價格': back_mean
        }
    
    grouped_random = merged_data.groupby(['出發日期', '航班代碼', '停留時間']).apply(
        lambda x: pd.Series(calculate_segmented_random_price(x)),
        include_groups=False
    ).reset_index()

    # 合併隨機購買平均價格到 grouped
    grouped = pd.merge(grouped, grouped_random, on=['出發日期', '航班代碼', '停留時間'], how='inner')

    # 計算最低價格的剩餘天數
    min_price_days = merged_data.loc[
        merged_data.groupby(['出發日期', '航班代碼', '停留時間'])['價格'].idxmin(),
        ['出發日期', '航班代碼', '停留時間', 'day left']
    ]
    min_price_days.rename(columns={'day left': '最低價格剩餘天數'}, inplace=True)
    
    # 合併統計數據與原資料
    final_data = pd.merge(merged_data.drop(columns=['價格', 'day left']),
                          grouped, on=['出發日期', '航班代碼', '停留時間'], how='inner')
    final_data = pd.merge(final_data, min_price_days, on=['出發日期', '航班代碼', '停留時間'], how='inner')
    final_data = pd.merge(final_data, valid_flights, on=['出發日期', '航班代碼', '停留時間'], how='inner')
    
    # 去除重複行，只保留每個航班的最後一筆資料，並排序
    final_data = final_data.sort_values(by=['出發日期', '航班代碼', '停留時間', '筆數']) \
                           .drop_duplicates(subset=['出發日期', '航班代碼', '停留時間'], keep='last')
    final_data = final_data.sort_values(by=['出發日期', '航班代碼（主航段）', '停留時間']).reset_index(drop=True)
    
    # (可選) 如需剔除平均價格離群值，可啟用下列程式
    # final_data = remove_outliers(final_data, '平均價格')
    
    # 確保輸出目錄存在
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)
    
    final_data.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"處理完成，{cabin_class} 資料儲存到 {output_path}")

#-------------------------------------------
# 主程式執行流程
base_path = '/Users/yuching/Documents/專題/web_scrapying/web_scrapying_data'

# 處理經濟艙資料
output_path_economy = '/Users/yuching/Documents/專題/merge_and_cleaned/data/long/sydney.csv'
process_and_merge_files(
    start_date='1021',
    end_date='0320',
    base_path=base_path,
    output_path=output_path_economy,
    file_pattern="sydney_{}.csv",
    cabin_class="economy"  # 僅保留經濟艙
)

# 處理商務艙資料
output_path_business = '/Users/yuching/Documents/專題/merge_and_cleaned/data/long/sydney_business.csv'
process_and_merge_files(
    start_date='1021',
    end_date='0320',
    base_path=base_path,
    output_path=output_path_business,
    file_pattern="sydney_business_{}.csv",
    cabin_class="business"  # 僅保留商務艙
)

開始處理，日期範圍：1021 到 0320
基礎路徑：/Users/yuching/Documents/專題/web_scrapying/web_scrapying_data
檔案格式：sydney_{}.csv
艙等篩選：economy
找到 151 個檔案，成功處理 151 個檔案
原始航班數量：12343，刪除少於 35 筆的航班數量：4897
處理完成，economy 資料儲存到 /Users/yuching/Documents/專題/merge_and_cleaned/data/long/sydney.csv
開始處理，日期範圍：1021 到 0320
基礎路徑：/Users/yuching/Documents/專題/web_scrapying/web_scrapying_data
檔案格式：sydney_business_{}.csv
艙等篩選：business
找到 151 個檔案，成功處理 151 個檔案
原始航班數量：11412，刪除少於 35 筆的航班數量：5260
處理完成，business 資料儲存到 /Users/yuching/Documents/專題/merge_and_cleaned/data/long/sydney_business.csv
